In [1]:
from scipy.io.arff import loadarff
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd
import scipy as sp

In [2]:
data, meta = loadarff('../data/KDDCup99_full.arff')

KeyboardInterrupt: 

In [3]:
meta

Dataset: kdd_cup_1999
	duration's type is numeric
	protocol_type's type is nominal, range is ('tcp', 'udp', 'icmp')
	service's type is nominal, range is ('vmnet', 'smtp', 'ntp_u', 'shell', 'kshell', 'aol', 'imap4', 'urh_i', 'netbios_ssn', 'tftp_u', 'mtp', 'uucp', 'nnsp', 'echo', 'tim_i', 'ssh', 'iso_tsap', 'time', 'netbios_ns', 'systat', 'hostnames', 'login', 'efs', 'supdup', 'http_8001', 'courier', 'ctf', 'finger', 'nntp', 'ftp_data', 'red_i', 'ldap', 'http', 'ftp', 'pm_dump', 'exec', 'klogin', 'auth', 'netbios_dgm', 'other', 'link', 'X11', 'discard', 'private', 'remote_job', 'IRC', 'daytime', 'pop_3', 'pop_2', 'gopher', 'sunrpc', 'name', 'rje', 'domain', 'uucp_path', 'http_2784', 'Z39_50', 'domain_u', 'csnet_ns', 'whois', 'eco_i', 'bgp', 'sql_net', 'printer', 'telnet', 'ecr_i', 'urp_i', 'netstat', 'http_443', 'harvest')
	flag's type is nominal, range is ('RSTR', 'S3', 'SF', 'RSTO', 'SH', 'OTH', 'S2', 'RSTOS0', 'S1', 'S0', 'REJ')
	src_bytes's type is numeric
	dst_bytes's type is numer

In [4]:
data = pd.DataFrame(np.asarray([list(i) for i in data], dtype=str))

Indices of columns to one hot vectorize with sk-learn:
    1, 2, 3, 41

In [5]:
cat_cols = [1,2,3,41]

Make variables numeric where needed

In [6]:
for x in data.columns:
    if x in cat_cols:
        continue
    data[x] = pd.to_numeric(data[x])

One-hot vectorize

In [7]:
data[41].unique()

array(['normal', 'buffer_overflow', 'loadmodule', 'perl', 'neptune',
       'smurf', 'guess_passwd', 'pod', 'teardrop', 'portsweep', 'ipsweep',
       'land', 'ftp_write', 'back', 'imap', 'satan', 'phf', 'nmap',
       'multihop', 'warezmaster', 'warezclient', 'spy', 'rootkit'], dtype=object)

In [8]:
for x in cat_cols:
    le = LabelEncoder()
    le.fit(data[x])
    data[x] = le.transform(data[x])

In [9]:
labels = data[41]
data.drop([41],inplace = True, axis = 1)

In [10]:
data.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40],
           dtype='int64')

In [11]:
labels.unique()

array([11,  1,  7, 12,  9, 18,  3, 14, 20, 15,  5,  6,  2,  0,  4, 17, 13,
       10,  8, 22, 21, 19, 16])

In [12]:
cat_cols = [1,2,3]

In [13]:
ohe = OneHotEncoder(sparse = False)
ohe.fit(data[cat_cols])
transformed = ohe.transform(data[cat_cols])

In [14]:
ohe = OneHotEncoder(sparse = False)
ohe.fit(labels.values.reshape(-1,1))
labels_k = ohe.transform(labels.values.reshape(-1,1))

In [15]:
transformed.shape

(4898431, 84)

In [16]:
labels.shape

(4898431,)

In [17]:
labels_k.shape

(4898431, 23)

In [18]:
data.drop(cat_cols, axis = 1, inplace = True)

In [19]:
data = data.values

In [20]:
data = np.concatenate((data, transformed), axis = 1)

In [30]:
#main_msk = np.random.uniform(size = data.shape[0]) < 0.7
#np.save('../data/main_splitting_mask.npy', main_msk)
main_msk = np.load('../data/main_splitting_mask.npy')

In [31]:
main_msk.shape

(4898431,)

In [32]:
X_train = data[main_msk, :]
y_train = labels[main_msk]
y_train_k = labels_k[main_msk]
data = data[~main_msk, :]
labels = labels[~main_msk]
labels_k = labels_k[~main_msk]

In [33]:
#sub_msk = np.random.uniform(size = data.shape[0]) < 0.5
#np.save('../data/sub_splitting_mask.npy', sub_msk)
sub_msk = np.load('../data/sub_splitting_mask.npy')

In [34]:
X_val = data[sub_msk, :]
y_val = labels[sub_msk]
y_val_k = labels_k[sub_msk]

In [35]:
np.save('../data/X_train.npy', X_train)
np.save('../data/y_train.npy', y_train)
np.save('../data/y_train_k.npy', y_train_k)
np.save('../data/X_val.npy', X_val)
np.save('../data/y_val.npy', y_val)
np.save('../data/y_val_k.npy', y_val_k)
np.save('../data/X_test.npy', data[~sub_msk,:])
np.save('../data/y_test.npy', labels[~sub_msk])
np.save('../data/y_test_k.npy', labels_k[~sub_msk])